In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import torch
from stable_diffusion3 import UniLatentPipeline, retrieve_timesteps
from diffusers import StableDiffusion3Pipeline as OGStableDiffusion3Pipeline

from diffusion.data.builder import build_dataset, build_dataloader
from diffusion.utils.data_sampler import AspectRatioBatchSampler
from torch.utils.data import RandomSampler

from tqdm import tqdm
from diffusers import get_cosine_schedule_with_warmup
from accelerate import Accelerator
from transformers import (
    GPT2Tokenizer,
    CLIPVisionModel,
    CLIPImageProcessor,
)

from caption_decoder import TextDecoder

/home/tiger/.local/lib/python3.9/site-packages/bytedmetrics/__init__.py:10: UserWarning: bytedmetrics is renamed to bytedance.metrics, please using `bytedance.metrics` instead of `bytedmetrics`
  warnings.warn("bytedmetrics is renamed to bytedance.metrics, please using `bytedance.metrics` instead of `bytedmetrics`")
/home/tiger/.local/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [3]:
pipe = UniLatentPipeline.from_pretrained('/mnt/bn/us-aigc-temp/henry/data/clip_test/', 
                    device_map=None, low_cpu_mem_usage=False, torch_dtype=torch.float32)
pipe.text_decoder = TextDecoder.from_pretrained('/mnt/bn/us-aigc-temp/henry/data/clip2text_2gpu/epoch_0_step_7499/', 
                        low_cpu_mem_usage=False, device_map=None)

<super: <class 'UniLatentPipeline'>, <UniLatentPipeline object>> ('/mnt/bn/us-aigc-temp/henry/data/clip_test/',) {'device_map': None, 'low_cpu_mem_usage': False, 'torch_dtype': torch.float32}


Loading pipeline components...:   0%|          | 0/11 [00:00<?, ?it/s]

Some weights of TextDecoder were not initialized from the model checkpoint at /mnt/bn/us-aigc-temp/henry/data/clip_test/text_decoder and are newly initialized: ['transformer.lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of TextDecoder were not initialized from the model checkpoint at /mnt/bn/us-aigc-temp/henry/data/clip2text_2gpu/epoch_0_step_7499/ and are newly initialized: ['transformer.lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
data_config = {
    'type': 'FlexibleInternalDataMS',
    'roots': [
        '/mnt/bn/us-aigc-temp/henry/coco_2014/val/val2014/',
        # '/mnt/bn/aigc-us/zjl/laion-coco-aesthetic/data_max1024/',
        # '/mnt/bn/aigc-us/zjl/openimages/data/',
        # '/mnt/bn/aigc-us/zjl/sharegpt4v_processed_data/data/'
    ],
    'json_lst': [
        '/mnt/bn/us-aigc-temp/henry/test.json',
    ],
    'load_vae_feat': False,
    'load_t5_feat': False
}
dataset = build_dataset(
    data_config, resolution=512, aspect_ratio_type='ASPECT_RATIO_512',
    real_prompt_ratio=0.0, max_length=77, return_image_id=True
)
batch_sampler = AspectRatioBatchSampler(sampler=RandomSampler(dataset), dataset=dataset,
                                    batch_size=1, aspect_ratios=dataset.aspect_ratio, drop_last=True,
                                    ratio_nums=dataset.ratio_nums, valid_num=0)
dataloader = build_dataloader(dataset, batch_sampler=batch_sampler, num_workers=10)

2024-07-24 00:47:07,504 - PixArt - INFO - Constructing dataset FlexibleInternalDataMS...
2024-07-24 00:47:07,505 - PixArt - INFO - T5 max token length: 77
2024-07-24 00:47:07,506 - PixArt - INFO - ratio of real user prompt: 0.0
2024-07-24 00:47:07,538 - PixArt - INFO - /mnt/bn/us-aigc-temp/henry/test.json data volume: 5000
2024-07-24 00:47:07,567 - PixArt - INFO - Dataset FlexibleInternalDataMS constructed. time: 0.06 s, length (use/ori): 5000/5000
2024-07-24 00:47:07,568 - PixArt - WARNING - Using valid_num=0 in config file. Available 40 aspect_ratios: ['0.25', '0.26', '0.27', '0.28', '0.32', '0.33', '0.35', '0.4', '0.42', '0.48', '0.5', '0.52', '0.57', '0.6', '0.68', '0.72', '0.78', '0.82', '0.88', '0.94', '1.0', '1.07', '1.13', '1.21', '1.29', '1.38', '1.46', '1.67', '1.75', '2.0', '2.09', '2.4', '2.5', '2.89', '3.0', '3.11', '3.62', '3.75', '3.88', '4.0']


In [5]:
from torch.nn.parallel import DistributedDataParallel as DDP
accelerator = Accelerator(
        mixed_precision='fp16',
    )

pipe.transformer = accelerator.prepare(pipe.transformer)
pipe.text_encoder, pipe.text_encoder_2 = accelerator.prepare(pipe.text_encoder, pipe.text_encoder_2)
pipe.clip_image_encoder, pipe.text_decoder = accelerator.prepare(pipe.clip_image_encoder, pipe.text_decoder)

Detected kernel version 5.4.143, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [6]:
iterloader = iter(dataloader)

In [10]:
len(dataloader)

5000

In [12]:
json_list = []
for i in range(len(dataloader)):
    batch = next(iterloader)

    with torch.no_grad():
        image_embd = pipe.encode_image(batch[0][:1], device=accelerator.device, dtype=torch.float16)
        generate_captions = pipe.text_decoder.module.generate_captions if isinstance(pipe.text_decoder, DDP) else pipe.text_decoder.generate_captions
        decoded_tokens = generate_captions(image_embd, 
                            eos_token_id=pipe.decoder_tokenizer.eos_token_id, device=accelerator.device)[0]
        decoded_text = pipe.decoder_tokenizer.batch_decode(decoded_tokens)[0]
    
    caption = decoded_text.strip('!').strip('<|endoftext|>').strip('<|EOS|>').strip()
    json_list.append({'image_id': batch[-1]['image_id'], 'caption': caption})
    print(f"Predicted: {caption} | True: {batch[1][0]}")

Predicted: A man riding on top of a skateboard down a street. | True: A man riding a skateboard across a street.
Predicted: A train is traveling down the tracks near some trees. | True: A train is moving along train tracks through the woods.
Predicted: People are riding on the bus at night. | True: People boarding a bus at a bus stop.
Predicted: A box filled with different types of donuts. | True: A box filled with six donuts separated by dividers.
Predicted: A train is on the tracks near a bridge. | True: A train as it travels down the tracks over a bridge.
Predicted: A plate with french toast and drinks on it. | True: Three slices of french toast with orange juice on a restaurant table
Predicted: A living room with shelves and pictures on the wall. | True: A television and some books in a room.
Predicted: Four plates with different types of food on them. | True: A group of four bowls sitting on top of a wooden table.
Predicted: A person on the back of a dirt bike. | True: A man ridin

StopIteration: 

In [ ]:
import json

save_path = '/mnt/bn/us-aigc-temp/henry/clip2text_captions.json'
with open(save_path, 'w') as f:
    test = json.dump(json_list, f)